# Classfication

In [66]:
import argparse
import os
import time
import os.path as osp
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

import torchvision
import torchvision.datasets as dset
import torchvision.transforms 
from torchvision import datasets 
from torchvision import transforms

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from torchcp.classification.predictors import ClusterPredictor, ClassWisePredictor, SplitPredictor,WeightedPredictor
from torchcp.classification.scores import THR, APS, SAPS, RAPS
from torchcp.classification import Metrics
from torchcp.utils import fix_randomness

## CovNet

In [12]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        # 第一个卷积层：输入通道数为3（彩色图像），输出通道数为4，卷积核大小为3x3
        self.conv1 = nn.Conv2d(3, 4, 3)

        # 最大池化层：窗口大小为2x2，步幅为2
        self.pool = nn.MaxPool2d(2, 2)

        # 第二个卷积层：输入通道数为4，输出通道数为8，卷积核大小为3x3
        self.conv2 = nn.Conv2d(4, 8, 3)

        # 全连接层1：输入特征维度为8x6x6，输出特征维度为32
        self.fc1 = nn.Linear(8 * 6 * 6, 32)

        # 全连接层2：输入特征维度为32，输出特征维度为10（分类数，通常对应着类别数量）
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # 第一卷积层 + 最大池化层 + 非线性激活函数 ReLU
        x = self.pool(torch.relu(self.conv1(x)))

        # 第二卷积层 + 最大池化层 + 非线性激活函数 ReLU
        x = self.pool(torch.relu(self.conv2(x)))

        # 将特征图展平成一维向量
        x = x.view(-1, 8 * 6 * 6)

        # 第一个全连接层 + 非线性激活函数 ReLU
        x = torch.relu(self.fc1(x))

        # 第二个全连接层，通常用于分类任务，输出类别分数
        x = self.fc2(x)

        return x

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ConvNet().to(device)

model_weights = torch.load("/data/lab/STA303-Exercise02/log/checkpoint_30.pth")
model.load_state_dict(model_weights,strict = False)
print(model)

ConvNet(
  (conv1): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)


## Dataset

In [14]:
BATCH_SIZE = 1024
# cifar10 transform，提前定义好对于训练数据集的处理。
transform_cifar10_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), #将照片裁剪，增加四个像素点来利于提取
    transforms.RandomHorizontalFlip(), #随机翻转一下提高随机性
    transforms.ToTensor(), # 转化为张量在gpu运行
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), # 归一化
]) #首先加载参数

#提前定义对于测试集的处理
transform_cifar10_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), 
])

train_set = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform_cifar10_train) #加载训练集，True
total_samples = len(train_set)
subset_size = 10000
subset_indices = random.sample(range(total_samples), subset_size)
subset_train_set = Subset(train_set, subset_indices)

train_dataloader = torch.utils.data.DataLoader(subset_train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2) #worker用于加速

test_set = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform_cifar10_test) #加载测试集，False

test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
len(train_dataloader.dataset)

10000

## SplitPredictor

In [52]:
def Score(preict):
    prediction_sets = []
    labels_list = []

    with torch.no_grad():
        for examples in tqdm(test_dataloader):
            tmp_x, tmp_label = examples[0], examples[1]
            prediction_sets_batch = predictor.predict(tmp_x)
            prediction_sets.extend(prediction_sets_batch)
            labels_list.append(tmp_label)

    test_labels = torch.cat(labels_list)
    alpha = 0.1
    num_classes = 10
    metrics = Metrics()
    print(f"Coverage_rate: {metrics('coverage_rate')(prediction_sets, test_labels)}, Average_size: {metrics('average_size')(prediction_sets, test_labels)}, CovGap: {metrics('CovGap')(prediction_sets, test_labels, alpha, num_classes)}.")


### THR

In [53]:
predictor = SplitPredictor(score_function=THR(), model=model)
predictor.calibrate(train_dataloader, alpha=0.1)
Score(predictor)

100%|██████████| 10/10 [00:01<00:00,  7.93it/s]

Coverage_rate: 0.9044, Average_size: 9.137, CovGap: 14.459999999999997.


### APS

In [54]:
predictor = SplitPredictor(score_function=APS(), model=model)
predictor.calibrate(train_dataloader, alpha=0.1)
Score(predictor)

100%|██████████| 10/10 [00:01<00:00,  7.59it/s]

Coverage_rate: 0.9045, Average_size: 9.0301, CovGap: 16.03.


### RAPS

In [55]:
ran = [0.1,0.3,0.5,0.7,0.9]
for m in ran:
    predictor = SplitPredictor(score_function=RAPS(m), model=model)
    predictor.calibrate(train_dataloader, alpha=0.1)
    print(f"RAPS: {m}")
    Score(predictor)

RAPS: 0.1


100%|██████████| 10/10 [00:01<00:00,  7.73it/s]

Coverage_rate: 0.8987, Average_size: 8.9836, CovGap: 16.109999999999996.


RAPS: 0.3


100%|██████████| 10/10 [00:01<00:00,  8.17it/s]


...

100%|██████████| 10/10 [00:01<00:00,  8.33it/s]

Coverage_rate: 0.9026, Average_size: 9.0083, CovGap: 16.319999999999997.


RAPS: 0.9


100%|██████████| 10/10 [00:01<00:00,  7.66it/s]

Coverage_rate: 0.8988, Average_size: 8.9769, CovGap: 16.14.


### SAPS

In [56]:
ran = [0.1,0.3,0.5,0.7,0.9]
for m in ran:
    predictor = SplitPredictor(score_function=SAPS(m), model=model)
    predictor.calibrate(train_dataloader, alpha=0.1)
    print(f"SAPS: {m}")
    Score(predictor)

SAPS: 0.1


100%|██████████| 10/10 [00:01<00:00,  7.92it/s]

Coverage_rate: 0.9, Average_size: 9.0062, CovGap: 16.06.


SAPS: 0.3


100%|██████████| 10/10 [00:01<00:00,  7.91it/s]

...

100%|██████████| 10/10 [00:01<00:00,  7.75it/s]

Coverage_rate: 0.9031, Average_size: 9.0165, CovGap: 16.33.


SAPS: 0.9


100%|██████████| 10/10 [00:01<00:00,  6.86it/s]

Coverage_rate: 0.9004, Average_size: 8.9884, CovGap: 16.28.


## ClassWisePredictor

### THR

In [57]:
predictor =ClassWisePredictor(score_function=THR(), model=model)
predictor.calibrate(train_dataloader, alpha=0.1)
Score(predictor)

100%|██████████| 10/10 [00:01<00:00,  7.98it/s]

Coverage_rate: 0.8995, Average_size: 9.0907, CovGap: 3.5100000000000007.


### APS

In [58]:
predictor = ClassWisePredictor(score_function=APS(), model=model)
predictor.calibrate(train_dataloader, alpha=0.1)
Score(predictor)

100%|██████████| 10/10 [00:01<00:00,  8.15it/s]

Coverage_rate: 0.9029, Average_size: 8.898, CovGap: 2.9300000000000015.


### RAPS

In [59]:
ran = [0.1,0.3,0.5,0.7,0.9]
for m in ran:
    predictor = ClassWisePredictor(score_function=RAPS(m), model=model)
    predictor.calibrate(train_dataloader, alpha=0.1)
    print(f"RAPS: {m}")
    Score(predictor)

RAPS: 0.1


100%|██████████| 10/10 [00:01<00:00,  8.16it/s]


Coverage_rate: 0.9037, Average_size: 8.8788, CovGap: 2.17.
RAPS: 0.3


100%|██████████| 10/10 [00:01<00:00,  8.30it/s]

Coverage_rate: 0.8993, Average_size: 8.8493, CovGap: 2.610000000000001.


...

RAPS: 0.7


100%|██████████| 10/10 [00:01<00:00,  8.29it/s]


Coverage_rate: 0.9052, Average_size: 8.8977, CovGap: 2.36.
RAPS: 0.9


100%|██████████| 10/10 [00:01<00:00,  7.91it/s]

Coverage_rate: 0.8984, Average_size: 8.84, CovGap: 2.460000000000001.


### SAPS

In [60]:
ran = [0.1,0.3,0.5,0.7,0.9]
for m in ran:
    predictor = ClassWisePredictor(score_function=SAPS(m), model=model)
    predictor.calibrate(train_dataloader, alpha=0.1)
    print(f"SAPS: {m}")
    Score(predictor)

SAPS: 0.1


100%|██████████| 10/10 [00:01<00:00,  7.73it/s]

Coverage_rate: 0.9027, Average_size: 8.8953, CovGap: 2.390000000000001.


SAPS: 0.3


100%|██████████| 10/10 [00:01<00:00,  7.98it/s]

...

100%|██████████| 10/10 [00:01<00:00,  7.80it/s]

Coverage_rate: 0.8954, Average_size: 8.8465, CovGap: 2.6.


SAPS: 0.9


100%|██████████| 10/10 [00:01<00:00,  7.70it/s]

Coverage_rate: 0.8933, Average_size: 8.7871, CovGap: 2.310000000000001.


## ClusterPredictor

### THR

In [61]:
predictor =ClusterPredictor(score_function=THR(), model=model)
predictor.calibrate(train_dataloader, alpha=0.1)
Score(predictor)

100%|██████████| 10/10 [00:01<00:00,  7.00it/s]

Coverage_rate: 0.904, Average_size: 9.1321, CovGap: 14.499999999999996.


### APS

In [63]:
predictor = ClusterPredictor(score_function=APS(), model=model)
predictor.calibrate(train_dataloader, alpha=0.1)
Score(predictor)

100%|██████████| 10/10 [00:01<00:00,  7.68it/s]

Coverage_rate: 0.9004, Average_size: 8.9916, CovGap: 16.08.


### RAPS

In [64]:
ran = [0.1,0.3,0.5,0.7,0.9]
for m in ran:
    predictor = ClusterPredictor(score_function=RAPS(m), model=model)
    predictor.calibrate(train_dataloader, alpha=0.1)
    print(f"RAPS: {m}")
    Score(predictor)

RAPS: 0.1


100%|██████████| 10/10 [00:01<00:00,  8.24it/s]

Coverage_rate: 0.9017, Average_size: 9.0004, CovGap: 16.409999999999997.


RAPS: 0.3


100%|██████████| 10/10 [00:01<00:00,  7.95it/s]


...

100%|██████████| 10/10 [00:01<00:00,  8.50it/s]

Coverage_rate: 0.901, Average_size: 8.9925, CovGap: 16.339999999999993.


RAPS: 0.9


100%|██████████| 10/10 [00:01<00:00,  7.80it/s]

Coverage_rate: 0.9017, Average_size: 9.0023, CovGap: 16.409999999999997.


### SAPS

In [65]:
ran = [0.1,0.3,0.5,0.7,0.9]
for m in ran:
    predictor = ClusterPredictor(score_function=SAPS(m), model=model)
    predictor.calibrate(train_dataloader, alpha=0.1)
    print(f"SAPS: {m}")
    Score(predictor)

SAPS: 0.1


100%|██████████| 10/10 [00:01<00:00,  8.02it/s]


Coverage_rate: 0.9046, Average_size: 9.0443, CovGap: 15.8.
SAPS: 0.3


100%|██████████| 10/10 [00:01<00:00,  7.89it/s]

Coverage_rate: 0.9019, Average_size: 9.0033, CovGap: 16.329999999999995.


...

100%|██████████| 10/10 [00:01<00:00,  7.51it/s]

Coverage_rate: 0.8977, Average_size: 8.9638, CovGap: 16.07.


SAPS: 0.9


100%|██████████| 10/10 [00:01<00:00,  7.50it/s]

Coverage_rate: 0.9015, Average_size: 8.9996, CovGap: 16.349999999999998.
